# 디렉토리에서 문서를 로드하는 방법

LangChain의 [DirectoryLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.directory.DirectoryLoader.html)는 디스크에서 파일을 읽어 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 객체로 변환하는 기능을 구현합니다.

이번 튜토리얼은 다음의 내용을 포함합니다.
- 와일드카드 패턴을 포함하여 파일 시스템에서 로드하는 방법
- 파일 I/O에 멀티스레딩을 사용하는 방법
- 특정 파일 유형(예: 코드)을 파싱하기 위해 사용자 정의 로더 클래스를 사용하는 방법
- 디코딩으로 인한 오류와 같은 오류를 처리하는 방법

## Jun---
- 현재 작업폴더는 어디일까?

In [ ]:
import os

print(os.getcwd())  # 현재 작업 디렉토리 확인

/content


**DirectoryLoader**는 기본적으로 [UnstructuredLoader](https://python.langchain.com/v0.2/docs/integrations/providers/unstructured/)
를 `loader_cls` kwarg로 받아들입니다. [Unstructured](https://unstructured-io.github.io/unstructured/)는 PDF와 HTML과 같은 다양한 형식의 파싱을 지원합니다. 여기서는 마크다운(.md) 파일을 읽기 위해 사용합니다.

`glob` 매개변수를 사용하여 로드할 파일 형식을 제어할 수 있습니다.

In [ ]:
!pip install -qU langchain_community
!pip install -qU unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=

## Jun---
- 상댕경로: ./와 ../ 설명
- *.md와 **/*.md 설명


In [ ]:
from langchain_community.document_loaders import DirectoryLoader

# 디렉토리 로더 초기화
loader = DirectoryLoader("./", glob="**/*.md")
# 문서 로드
docs = loader.load()
# 문서 개수 계산
len(docs)

2

In [ ]:
# 페이지 내용 출력
print(docs[1].page_content[:100])

This directory includes a few sample datasets to get you started.

california_housing_data*.csv is C


기본적으로 진행 상태 표시줄은 표시되지 않습니다. `show_progress=True` 옵션으로 진행상황을 확인할 수 있습니다.

**참고**
- 진행 상태 표시줄을 표시하려면 `tqdm` 라이브러리를 설치(예: `pip install tqdm`)
- `show_progress` 매개변수를 `True`로 설정

In [ ]:
!pip install -qU tqdm

In [ ]:
loader = DirectoryLoader(
    "./", glob="**/*.md", show_progress=True
)  # 디렉토리 로더 설정
docs = loader.load()  # 문서 로드

100%|██████████| 2/2 [00:00<00:00, 12.27it/s]


기본적으로 로딩은 하나의 스레드에서 이루어집니다.

만약, 여러 스레드를 활용하려면 `use_multithreading` 플래그를 true로 설정하세요.

In [ ]:
loader = DirectoryLoader(
    "./", glob="**/*.md", use_multithreading=True
)  # 디렉토리 로더 설정
docs = loader.load()  # 문서 로드

## loader_cls 변경

`loader_cls` 는 기본 값으로 `UnstructuredLoader` 클래스를 사용합니다.

로더를 사용자 정의하려면 `loader_cls` kwarg에 로더 클래스를 지정하세요.

In [ ]:
from langchain_community.document_loaders import TextLoader

# loader_cls 를 TextLoader 로 변경합니다.
loader = DirectoryLoader("/content/", glob="**/*.md", loader_cls=TextLoader)

# 문서 로드
docs = loader.load()

In [ ]:
# 문서 페이지 내용 출력
print(docs[0].page_content[:100])

알버트 아인슈타인 자기소개서

성장 배경 및 학업 과정

안녕하십니까. 저는 **알버트 아인슈타인(Albert Einstein)**이라고 합니다. 1879년 독일 울름에서 태어나,


---

`UnstructuredLoader`는 Markdown 헤더를 파싱하는 반면, `TextLoader`는 그렇지 않습니다.

Python 소스 코드 파일을 로드해야 하는 경우, `PythonLoader`를 사용하세요.

In [ ]:
from langchain_community.document_loaders import PythonLoader

# 현재폴더(.) 의 .py 파일을 모두 조회하여 PythonLoader 로 로드
loader = DirectoryLoader("./", glob="**/*.py", loader_cls=PythonLoader)

In [ ]:
# 문서 로드
docs = loader.load()
docs